# BITSEI QUERIES

DISCLAIMER! If you want to execute the queries, run this code while your computer is connected to the internet from any of the University of Padova buildings!

(remote server allows connections from the IP class 147.162.0.0\16)

In [3]:
from SPARQLWrapper import SPARQLWrapper, CSV, SPARQLExceptions
from io import StringIO 
import pandas as pd

pd.set_option('display.max_rows', 10)
sparql = SPARQLWrapper("http://bitsei.it:7200/repositories/LARepo")
sparql.setReturnFormat(CSV)
#sparql.setTimeout(10000)

In [4]:
def executeQuery(query):
	sparql.setQuery(query)

	executed = False
	while(not executed):
			try:
					ret = sparql.queryAndConvert().decode("utf-8")
					csvStringIO = StringIO(ret)
					readed = pd.read_csv(csvStringIO, sep=",")
					

					display(readed)
					executed = True
			except Exception as e :
					if(type(e) != SPARQLExceptions.EndPointInternalError):
							print(e)
							executed=True

### TODO Query 1: Number of: COVID cases, Opened and Closed Businesses, and Crime Events grouped by month and year

### Query 1A: Number of COVID cases and deaths grouped by month and year

In [177]:

query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (SUM(?newCases) AS ?numOfCovidCases) (ROUND((?numOfCovidCases*100 / 30))/100 AS ?casesRatio) (SUM(?newDeaths) AS ?numOfDeaths) (ROUND((?numOfDeaths*100 / 30))/100 AS ?deathsRatio)
WHERE {
    ?day lao:hasDate ?date ;
         lao:hasNewCases ?newCases ;
         lao:hasNewDeaths ?newDeaths .
         
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)""";


executeQuery(query)

,period,numOfCovidCases,casesRatio,numOfDeaths,deathsRatio
0,2020-01,0,0.00,0,0.00
1,2020-02,4045,134.83,78,2.60
2,2020-03,-1026,-34.20,-24,-0.80
3,2020-04,19466,648.87,1002,33.40
4,2020-05,31261,1042.03,1283,42.77
...,...,...,...,...,...
31,2022-08,112492,3749.73,430,14.33
32,2022-09,49358,1645.27,460,15.33
33,2022-10,31245,1041.50,353,11.77
34,2022-11,52222,1740.73,228,7.60


### Query 1B: Number of Opened Businesses grouped by month and year

In [178]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (COUNT(?openBusiness) AS ?openedBusinesses) (ROUND((?openedBusinesses*100 / 30))/100 AS ?ratio)
WHERE {
    ?openBusiness lao:openedOnDate ?day .
    ?day lao:hasDate ?date .
    
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)""";

executeQuery(query)

,period,openedBusinesses,ratio
0,2020-01,10565,352.17
1,2020-02,3456,115.20
2,2020-03,2642,88.07
3,2020-04,1984,66.13
4,2020-05,1898,63.27
...,...,...,...
31,2022-08,2289,76.30
32,2022-09,2056,68.53
33,2022-10,2204,73.47
34,2022-11,1744,58.13


### Query 1C: Number of Closed Businesses grouped by month and year

In [179]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (COUNT(?closeBusiness) AS ?closedBusinesses) (ROUND(?closedBusinesses*100 / 30)/100 AS ?ratio)
WHERE {
    ?closeBusiness lao:closedOnDate ?day .
    ?day lao:hasDate ?date .
    
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)""";

executeQuery(query)

,period,closedBusinesses,ratio
0,2020-01,3343,111.43
1,2020-02,1529,50.97
2,2020-03,2896,96.53
3,2020-04,1052,35.07
4,2020-05,954,31.80
...,...,...,...
31,2022-08,733,24.43
32,2022-09,725,24.17
33,2022-10,668,22.27
34,2022-11,655,21.83


### Query 1D: Number of Crime Events grouped by month and year

In [180]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (COUNT(?crimeEvent) AS ?crimeEvents) (ROUND(?crimeEvents*100 / 30)/100 AS ?ratio)
WHERE {
    ?crimeEvent lao:occurredOnDate ?day .
    ?day lao:hasDate ?date .
    
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)""";


executeQuery(query)

,period,crimeEvents,ratio
0,2020-01,18487,616.23
1,2020-02,17252,575.07
2,2020-03,16162,538.73
3,2020-04,15678,522.60
4,2020-05,17200,573.33
...,...,...,...
31,2022-08,20073,669.10
32,2022-09,19268,642.27
33,2022-10,20252,675.07
34,2022-11,18666,622.20


### Query 2A: Number of COVID Cases and Deaths grouped by period, with ratio on period duration

In [181]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (SUM(?newCases) AS ?numOfCovidCases) (ROUND((?numOfCovidCases*100 / ?duration))/100 AS ?casesRatio) (SUM(?newDeaths) AS ?numOfDeaths) (ROUND((?numOfDeaths*100 / ?duration))/100 AS ?deathsRatio)
WHERE {
    ?day lao:hasDate ?date ;
         lao:hasNewCases ?newCases ;
         lao:hasNewDeaths ?newDeaths ;
         lao:belongsTo ?period .
         
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed """;


executeQuery(query)

,periodLabel,startDateParsed,endDateParsed,duration,numOfCovidCases,casesRatio,numOfDeaths,deathsRatio
0,Covid Starting Period,2020-01-01,2020-03-10,69,4045,58.62,78,1.13
1,First Lockdown Period,2020-03-11,2020-05-28,78,43800,561.54,2067,26.50
2,First Reopening Period,2020-05-29,2020-11-16,171,287910,1683.68,5066,29.63
3,Mask Mandate Period,2020-11-17,2021-01-24,68,715232,10518.12,7625,112.13
4,Restrictions Eased Period,2021-01-25,2021-04-09,74,157700,2131.08,8160,110.27
...,...,...,...,...,...,...,...,...
6,Restrictions Dropped Period,2021-06-15,2021-10-01,108,211543,1958.73,1644,15.22
7,Vaccine Requirement Period,2021-10-02,2022-01-04,94,282110,3001.17,1534,16.32
8,Mask Mandate Extended Period,2022-01-05,2022-01-26,21,433197,20628.43,657,31.29
9,Executive Orders Dropped Period,2022-01-27,2022-12-01,308,759992,2467.51,3807,12.36


### Query 2B: Number of Opened Businesses grouped by period, with ratio on period duration

In [182]:
query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (COUNT(?openBusiness) AS ?openedBusinesses) (ROUND(?openedBusinesses*100/?duration)/100 AS ?openedRatio)
WHERE {
    ?openBusiness lao:openedOnDate ?day .
    ?day lao:hasDate ?date ;
         lao:belongsTo ?period .
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed """;

executeQuery(query)

,periodLabel,startDateParsed,endDateParsed,duration,openedBusinesses,openedRatio
0,Covid Starting Period,2020-01-01,2020-03-10,69,9136,132.41
1,First Lockdown Period,2020-03-11,2020-05-28,78,4750,60.90
2,First Reopening Period,2020-05-29,2020-11-16,171,14230,83.22
3,Mask Mandate Period,2020-11-17,2021-01-24,68,9995,146.99
4,Restrictions Eased Period,2021-01-25,2021-04-09,74,7265,98.18
...,...,...,...,...,...,...
6,Restrictions Dropped Period,2021-06-15,2021-10-01,108,8810,81.57
7,Vaccine Requirement Period,2021-10-02,2022-01-04,94,10441,111.07
8,Mask Mandate Extended Period,2022-01-05,2022-01-26,21,974,46.38
9,Executive Orders Dropped Period,2022-01-27,2022-12-01,308,20852,67.70


### Query 2C: Number of Closed Businesses grouped by period, with ratio on period duration

In [183]:
query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (COUNT(?closeBusiness) AS ?closedBusinesses) (ROUND(?closedBusinesses*100/?duration)/100 AS ?closedRatio)
WHERE {
    ?closeBusiness lao:closedOnDate ?day .
    ?day lao:hasDate ?date ;
         lao:belongsTo ?period .
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed """;

executeQuery(query)

,periodLabel,startDateParsed,endDateParsed,duration,closedBusinesses,closedRatio
0,Covid Starting Period,2020-01-01,2020-03-10,69,3579,51.87
1,First Lockdown Period,2020-03-11,2020-05-28,78,3614,46.33
2,First Reopening Period,2020-05-29,2020-11-16,171,7152,41.82
3,Mask Mandate Period,2020-11-17,2021-01-24,68,18208,267.76
4,Restrictions Eased Period,2021-01-25,2021-04-09,74,2661,35.96
...,...,...,...,...,...,...
6,Restrictions Dropped Period,2021-06-15,2021-10-01,108,3834,35.50
7,Vaccine Requirement Period,2021-10-02,2022-01-04,94,10745,114.31
8,Mask Mandate Extended Period,2022-01-05,2022-01-26,21,185,8.81
9,Executive Orders Dropped Period,2022-01-27,2022-12-01,308,6735,21.87


### Query 2D: Number of Crime Events grouped by period, with ratio on period duration

In [184]:
query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (COUNT(?crimeEvent) AS ?crimeEvents) (ROUND(?crimeEvents*100/?duration)/100 AS ?crimeEventsRatio)
WHERE {
    ?crimeEvent lao:occurredOnDate ?day .
    ?day lao:hasDate ?date ;
         lao:belongsTo ?period .
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed """;

executeQuery(query)

,periodLabel,startDateParsed,endDateParsed,duration,crimeEvents,crimeEventsRatio
0,Covid Starting Period,2020-01-01,2020-03-10,69,39893,578.16
1,First Lockdown Period,2020-03-11,2020-05-28,78,39916,511.74
2,First Reopening Period,2020-05-29,2020-11-16,171,93038,544.08
3,Mask Mandate Period,2020-11-17,2021-01-24,68,35776,526.12
4,Restrictions Eased Period,2021-01-25,2021-04-09,74,39604,535.19
...,...,...,...,...,...,...
6,Restrictions Dropped Period,2021-06-15,2021-10-01,108,64642,598.54
7,Vaccine Requirement Period,2021-10-02,2022-01-04,94,56911,605.44
8,Mask Mandate Extended Period,2022-01-05,2022-01-26,21,7149,340.43
9,Executive Orders Dropped Period,2022-01-27,2022-12-01,308,181319,588.70


### Query 3A: Number of Opened Businesses grouped by area, with ratio on area surface

In [185]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?openBusiness) AS ?openedBusinesses) (((ROUND(?openedBusinesses * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?openBusiness lao:openedOnDate ?day ;
                  lao:locatedIn ?location .
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface
ORDER BY DESC(?ratio)""";

executeQuery(query)

,areaName,areaAcronym,areaSize,openedBusinesses,ratio
0,Central City,CCY,13.19,2552,193.482
1,Westlake,WLK,11.48,1195,104.051
2,Wilshire,WIL,52.98,4806,90.714
3,West Los Angeles,WLA,27.04,1837,67.941
4,Van Nuys,VNY,48.80,2688,55.079
...,...,...,...,...,...
32,Sunland,SLD,97.34,542,5.568
33,Brentwood,BTW,144.04,796,5.526
34,Bel Air,BAR,57.30,217,3.787
35,Los Angeles International Airport,LAX,23.57,37,1.570


### Query 3B: Number of Closed Businesses grouped by area, with ratio on area surface

In [186]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?closeBusiness) AS ?closedBusinesses) (((ROUND(?closedBusinesses * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?closeBusiness lao:closedOnDate ?day ;
                  lao:locatedIn ?location .
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface
ORDER BY DESC(?ratio)""";

executeQuery(query)


,areaName,areaAcronym,areaSize,closedBusinesses,ratio
0,Central City,CCY,13.19,754,57.165
1,Wilshire,WIL,52.98,1033,19.498
2,Westlake,WLK,11.48,214,18.633
3,West Los Angeles,WLA,27.04,349,12.908
4,Venice,VEN,11.84,139,11.744
...,...,...,...,...,...
32,Brentwood,BTW,144.04,150,1.041
33,Sunland,SLD,97.34,97,0.996
34,Bel Air,BAR,57.30,37,0.646
35,Los Angeles International Airport,LAX,23.57,12,0.509


### Query 3C: Number of Crime Events grouped by area, with ratio on area surface

In [187]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location .
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface
ORDER BY DESC(?ratio)""";
executeQuery(query)




,areaName,areaAcronym,areaSize,crimeEvents,ratio
0,Central City,CCY,13.19,34549,2619.355
1,Westlake,WLK,11.48,20029,1743.968
2,Southeast Los Angeles,SEL,57.42,53503,931.856
3,Wilshire,WIL,52.98,48498,915.410
4,South Los Angeles,SLA,59.53,53529,899.141
...,...,...,...,...,...
32,Granada Hills,GHL,61.66,5536,89.778
33,Sunland,SLD,97.34,5373,55.196
34,Brentwood,BTW,144.04,5205,36.136
35,Bel Air,BAR,57.30,1607,28.045


### Query 3D: Number of Violent Crime Events grouped by area, with ratio on area surface

In [188]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?violentCrimeEvents)  (((ROUND(?violentCrimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:ViolentCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)""";

executeQuery(query)


,areaName,areaAcronym,areaSize,violentCrimeEvents,ratio
0,Central City,CCY,13.19,3751,284.385
1,Central City,CCY,13.19,2485,188.402
2,Westlake,WLK,11.48,1993,173.535
3,Westlake,WLK,11.48,1493,129.999
4,Central City,CCY,13.19,1678,127.219
...,...,...,...,...,...
747,Encino,ENC,76.96,1,0.013
748,Hollywood,HWD,95.06,1,0.011
749,Northeast Los Angeles,NLA,92.78,1,0.011
750,Sunland,SLD,97.34,1,0.010


### Query 3E: Number of Sexual Crime Events grouped by area, with ratio on area surface

In [189]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?sexualCrimeEvents) (((ROUND(?sexualCrimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:SexualCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)""";

executeQuery(query)


,areaName,areaAcronym,areaSize,sexualCrimeEvents,ratio
0,Central City,CCY,13.19,125,9.477
1,Westlake,WLK,11.48,62,5.398
2,Westlake,WLK,11.48,42,3.657
3,South Los Angeles,SLA,59.53,216,3.628
4,Southeast Los Angeles,SEL,57.42,198,3.449
...,...,...,...,...,...
285,Encino,ENC,76.96,1,0.013
286,Northeast Los Angeles,NLA,92.78,1,0.011
287,Hollywood,HWD,95.06,1,0.011
288,Sunland,SLD,97.34,1,0.010


### Query 3F: Number of Property Crime Events grouped by area, with ratio on area surface

In [190]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?propertyCrimeEvents) (((ROUND(?propertyCrimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:PropertyCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)""";
executeQuery(query)

,areaName,areaAcronym,areaSize,propertyCrimeEvents,ratio
0,Central City,CCY,13.19,4555,345.340
1,Westlake,WLK,11.48,2063,179.630
2,Central City,CCY,13.19,2211,167.628
3,Central City,CCY,13.19,2010,152.389
4,Southeast Los Angeles,SEL,57.42,7548,131.463
...,...,...,...,...,...
697,Northeast Los Angeles,NLA,92.78,1,0.011
698,Chatsworth,CHT,92.29,1,0.011
699,Sunland,SLD,97.34,1,0.010
700,Canoga Park,CPK,106.18,1,0.009


### Query 3G: Number of White Collar Crime Events grouped by area, with ratio on area surface

In [191]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?whiteCollarCrimeEvents) (((ROUND(?whiteCollarCrimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:WhiteCollarCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)""";
executeQuery(query)


,areaName,areaAcronym,areaSize,whiteCollarCrimeEvents,ratio
0,Westlake,WLK,11.48,1392,121.204
1,Central City,CCY,13.19,953,72.252
2,Southeast Los Angeles,SEL,57.42,3936,68.553
3,West Adams,WAD,48.50,2746,56.615
4,South Los Angeles,SLA,59.53,3223,54.138
...,...,...,...,...,...
386,Chatsworth,CHT,92.29,1,0.011
387,Hollywood,HWD,95.06,1,0.011
388,Sunland,SLD,97.34,1,0.010
389,Canoga Park,CPK,106.18,1,0.009


### Query 3H: Number of Public Order Crime Events grouped by area, with ratio on area surface

In [192]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?publicOrderCrimeEvents) (((ROUND(?publicOrderCrimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:PublicOrderCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)""";

executeQuery(query)

,areaName,areaAcronym,areaSize,publicOrderCrimeEvents,ratio
0,Central City,CCY,13.19,2499,189.463
1,Westlake,WLK,11.48,1222,106.402
2,Central City,CCY,13.19,839,63.609
3,Venice,VEN,11.84,701,59.228
4,Wilshire,WIL,52.98,3033,57.249
...,...,...,...,...,...
655,Hollywood,HWD,95.06,1,0.011
656,Northeast Los Angeles,NLA,92.78,1,0.011
657,Sunland,SLD,97.34,1,0.010
658,Canoga Park,CPK,106.18,1,0.009


### Query 4A: Number of Opened Businesses grouped by NAICS code

In [193]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?naicsDesc (COUNT(?openBusiness) AS ?openedBusinesses) (ROUND(?openedBusinesses*1000 / ?totBusinesses)/1000 AS ?ratio) 
WHERE {
    ?openBusiness lao:openedOnDate ?day ;
                  lao:hasNaics ?naics .
    ?day lao:hasDate ?date .
    ?naics lao:naicsCode ?naicsCode ;
           lao:naicsDescription ?naicsDesc .
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)

    {
        SELECT DISTINCT (COUNT(?openBus) AS ?totBusinesses) 
        WHERE {
            ?openBus lao:openedOnDate ?openDay .
            ?openDay lao:hasDate ?openDate .
            FILTER (xsd:date(?openDate) >= \"2020-01-01\"^^xsd:date && xsd:date(?openDate) <= \"2022-12-31\"^^xsd:date)
        }                        
    }
}
GROUP BY ?naicsCode ?naicsDesc ?totBusinesses
ORDER BY DESC(?openedBusinesses)""";

executeQuery(query)


,naicsDesc,openedBusinesses,ratio
0,All Other Personal Services,8508,0.081
1,"Independent Artists, Writers, and Performers",4379,0.042
2,All Other Specialty Trade Contractors,2138,0.020
3,Janitorial Services,1958,0.019
4,"All Other Professional, Scientific, and Techni...",1933,0.018
...,...,...,...
91,Recreational Vehicle Dealers,2,0.000
92,Boat Dealers,2,0.000
93,Securities and Commodity Exchanges,2,0.000
94,Interurban and Rural Bus Transportation,1,0.000


### Query 4B: Number of Closed Businesses grouped by NAICS code

In [194]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?naicsDesc (COUNT(?closeBusiness) AS ?closedBusinesses) (ROUND(?closedBusinesses*1000 / ?totBusinesses)/1000 AS ?ratio) 
WHERE {
    ?closeBusiness lao:closedOnDate ?day ;
                  lao:hasNaics ?naics .
    ?day lao:hasDate ?date .
    ?naics lao:naicsCode ?naicsCode ;
           lao:naicsDescription ?naicsDesc .
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT(\"00\", ?year) AS ?paddedYear)
    BIND(CONCAT(\"0000\", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, \"-\", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)

    {
        SELECT DISTINCT (COUNT(?closeBus) AS ?totBusinesses) 
        WHERE {
            ?closeBus lao:closedOnDate ?closeDay .
            ?closeDay lao:hasDate ?closeDate .		
            FILTER (xsd:date(?closeDate) >= \"2020-01-01\"^^xsd:date && xsd:date(?closeDate) <= \"2022-12-31\"^^xsd:date)
        }                        
    }
}
GROUP BY ?naicsCode ?naicsDesc ?totBusinesses
ORDER BY DESC(?closedBusinesses)""";

executeQuery(query)


,naicsDesc,closedBusinesses,ratio
0,All Other Personal Services,800,0.012
1,"Independent Artists, Writers, and Performers",376,0.006
2,"All Other Professional, Scientific, and Techni...",267,0.004
3,Janitorial Services,241,0.004
4,All Other Specialty Trade Contractors,162,0.002
...,...,...,...
83,Surveying and Mapping (except Geophysical) Ser...,1,0.000
84,Payroll Services,1,0.000
85,Testing Laboratories and Services,1,0.000
86,Offices of Real Estate Appraisers,1,0.000


### Query 5A: Number of Crime Events grouped by crime category

In [195]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?crimeCategory (COUNT(?crimeEvent) AS ?numOfCrimeEvents) 
WHERE {
    ?crimeEvent lao:isOfType ?crime .
    ?crime a ?crimeCat .
    ?crimeCat rdfs:subClassOf lao:Crime .
    BIND(REPLACE(STR(?crimeCat), \".*/\", \"\") AS ?crimeCategory)
}
GROUP BY(?crimeCategory)
ORDER BY DESC(?numOfCrimeEvents)""";

executeQuery(query)

,crimeCategory,numOfCrimeEvents
0,PropertyCrime,368831
1,ViolentCrime,238996
2,PublicOrderCrime,137743
3,WhiteCollarCrime,74291
4,SexualCrime,8159


### Query 5B: Number of Crime Events grouped by period and crime category

In [196]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?periodLabel 
       (SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed) 
       (SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed) 
       (COUNT(?sexualCrimeEvent) AS ?numSexualCrimes)
       (COUNT(?propertyCrimeEvent) AS ?numPropertyCrimes)
       (COUNT(?whiteCollarCrimeEvent) AS ?numWhiteCollarCrimes)
       (COUNT(?violentCrimeEvent) AS ?numViolentCrimes)
       (COUNT(?publicOrderCrimeEvent) AS ?numPublicOrderCrimes)
WHERE {
  ?crimeEvent lao:occurredOnDate ?day ;
              lao:isOfType ?crime .
  ?day lao:hasDate ?date ;
       lao:belongsTo ?period .
  ?period lao:hasLabel ?periodLabel ;
          lao:hasStartDate ?startDate ;
          lao:hasEndDate ?endDate.

  FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)

  OPTIONAL { ?crime a lao:SexualCrime . BIND(?crimeEvent AS ?sexualCrimeEvent) }
  OPTIONAL { ?crime a lao:PropertyCrime . BIND(?crimeEvent AS ?propertyCrimeEvent) }
  OPTIONAL { ?crime a lao:WhiteCollarCrime . BIND(?crimeEvent AS ?whiteCollarCrimeEvent) }
  OPTIONAL { ?crime a lao:ViolentCrime . BIND(?crimeEvent AS ?violentCrimeEvent) }
  OPTIONAL { ?crime a lao:PublicOrderCrime . BIND(?crimeEvent AS ?publicOrderCrimeEvent) }
}
GROUP BY ?periodLabel ?startDate ?endDate
ORDER BY (?startDate)
""";

executeQuery(query)

,periodLabel,startDateParsed,endDateParsed,numSexualCrimes,numPropertyCrimes,numWhiteCollarCrimes,numViolentCrimes,numPublicOrderCrimes
0,Covid Starting Period,2020-01-01,2020-03-10,540,17653,3281,11329,7025
1,First Lockdown Period,2020-03-11,2020-05-28,430,17959,2682,11568,7227
2,First Reopening Period,2020-05-29,2020-11-16,1045,39134,6284,29224,17081
3,Mask Mandate Period,2020-11-17,2021-01-24,354,16251,2595,10268,6166
4,Restrictions Eased Period,2021-01-25,2021-04-09,404,17575,3263,11280,6989
...,...,...,...,...,...,...,...,...
6,Restrictions Dropped Period,2021-06-15,2021-10-01,627,27252,5033,19953,11627
7,Vaccine Requirement Period,2021-10-02,2022-01-04,606,29051,5497,17312,9864
8,Mask Mandate Extended Period,2022-01-05,2022-01-26,46,3384,506,2075,1123
9,Executive Orders Dropped Period,2022-01-27,2022-12-01,1737,79217,21616,50716,27650


### Query 5C: Number of Crime Events grouped by area and crime category

In [199]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT 
?areaName ?areaAcronym 
(COUNT(?crimeEvent) AS ?totNumOfCrimes)
(COUNT(?sexualCrimeEvent) AS ?numSexualCrimes)
(COUNT(?propertyCrimeEvent) AS ?numPropertyCrimes)
(COUNT(?whiteCollarCrimeEvent) AS ?numWhiteCollarCrimes)
(COUNT(?violentCrimeEvent) AS ?numViolentCrimes)
(COUNT(?publicOrderCrimeEvent) AS ?numPublicOrderCrimes)
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
              lao:isOfType ?crime ;
              lao:occurredInLocation ?location .
  
    ?location lao:belongsToArea ?area .
    
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym .
    
    ?day lao:hasDate ?date ;
       lao:belongsTo ?period .
  
    ?period lao:hasLabel ?periodLabel ;
          lao:hasStartDate ?startDate ;
          lao:hasEndDate ?endDate.
    
    FILTER (xsd:date(?date) >= \"2020-01-01\"^^xsd:date && xsd:date(?date) <= \"2022-12-31\"^^xsd:date)
  
    OPTIONAL { ?crime a lao:SexualCrime . BIND(?crimeEvent AS ?sexualCrimeEvent) }
    OPTIONAL { ?crime a lao:PropertyCrime . BIND(?crimeEvent AS ?propertyCrimeEvent) }
    OPTIONAL { ?crime a lao:WhiteCollarCrime . BIND(?crimeEvent AS ?whiteCollarCrimeEvent) }
    OPTIONAL { ?crime a lao:ViolentCrime . BIND(?crimeEvent AS ?violentCrimeEvent) }
    OPTIONAL { ?crime a lao:PublicOrderCrime . BIND(?crimeEvent AS ?publicOrderCrimeEvent) }
}
GROUP BY ?areaName ?areaAcronym
ORDER BY DESC(?totNumOfCrimes)""";

executeQuery(query)

,areaName,areaAcronym,totNumOfCrimes,numSexualCrimes,numPropertyCrimes,numWhiteCollarCrimes,numViolentCrimes,numPublicOrderCrimes
0,South Los Angeles,SLA,51259,769,18068,3850,19642,8623
1,Southeast Los Angeles,SEL,51162,707,17124,4205,20174,8661
2,Wilshire,WIL,46278,482,21968,3935,12569,7300
3,Hollywood,HWD,41482,378,18910,3290,11746,7152
4,Central City,CCY,32943,320,14773,1428,11485,4902
...,...,...,...,...,...,...,...,...
32,Brentwood,BTW,4979,36,2828,765,622,726
33,Westwood,WWD,3923,22,2010,486,753,653
34,Los Angeles International Airport,LAX,3359,14,1814,281,525,723
35,Bel Air,BAR,1535,15,703,249,236,333


## Query 6
### Get the number of crimes (for each period), grouped by the modus operandi involved

In [200]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>

SELECT ?moDesc
       (SUM(?crimeCovidStartingPeriod) AS ?crimeCovidStartingPeriod)
       (SUM(?crimeFirstLockdownPeriod) AS ?crimeFirstLockdownPeriod)
       (SUM(?crimeFirstReopeningPeriod) AS ?crimeFirstReopeningPeriod)
       (SUM(?crimeMaskMandatePeriod) AS ?crimeMaskMandatePeriod)
       (SUM(?crimeRestrictionsEasedPeriod) AS ?crimeRestrictionsEasedPeriod)
       (SUM(?crimeVaccineAvailabilityPeriod) AS ?crimeVaccineAvailabilityPeriod)
       (SUM(?crimeRestrictionsDroppedPeriod) AS ?crimeRestrictionsDroppedPeriod)
       (SUM(?crimeVaccineRequirementPeriod) AS ?crimeVaccineRequirementPeriod)
       (SUM(?crimeMaskMandateExtendedPeriod) AS ?crimeMaskMandateExtendedPeriod)
       (SUM(?crimeSecondRestrictionsEasedPeriod) AS ?crimeSecondRestrictionsEasedPeriod)
       (SUM(?crimeExecutiveOrdersDroppedPeriod) AS ?crimeExecutiveOrdersDroppedPeriod)
WHERE {
  ?mo lao:moDesc ?moDesc .
  ?crimeEvent lao:hasModusOperandi ?mo ;
              lao:occurredOnDate ?day .
  ?day lao:belongsTo ?period .
  ?period lao:hasLabel ?label .

  BIND(IF(?label = \"Covid Starting Period\", 1, 0) AS ?crimeCovidStartingPeriod)
  BIND(IF(?label = \"First Lockdown Period\", 1, 0) AS ?crimeFirstLockdownPeriod)
  BIND(IF(?label = \"First Reopening Period\", 1, 0) AS ?crimeFirstReopeningPeriod)
  BIND(IF(?label = \"Mask Mandate Period\", 1, 0) AS ?crimeMaskMandatePeriod)
  BIND(IF(?label = \"Restrictions Eased Period\", 1, 0) AS ?crimeRestrictionsEasedPeriod)
  BIND(IF(?label = \"Vaccine Availability Period\", 1, 0) AS ?crimeVaccineAvailabilityPeriod)
  BIND(IF(?label = \"Restrictions Dropped Period\", 1, 0) AS ?crimeRestrictionsDroppedPeriod)
  BIND(IF(?label = \"Vaccine Requirement Period\", 1, 0) AS ?crimeVaccineRequirementPeriod)
  BIND(IF(?label = \"Mask Mandate Extended Period\", 1, 0) AS ?crimeMaskMandateExtendedPeriod)
  BIND(IF(?label = \"Second Restrictions Eased Period\", 1, 0) AS ?crimeSecondRestrictionsEasedPeriod)
  BIND(IF(?label = \"Executive Orders Dropped Period\", 1, 0) AS ?crimeExecutiveOrdersDroppedPeriod)
}
GROUP BY ?moDesc
ORDER BY DESC(?crimeCovidStartingPeriod)""";

executeQuery(query)

,moDesc,crimeCovidStartingPeriod,crimeFirstLockdownPeriod,crimeFirstReopeningPeriod,crimeMaskMandatePeriod,crimeRestrictionsEasedPeriod,crimeVaccineAvailabilityPeriod,crimeRestrictionsDroppedPeriod,crimeVaccineRequirementPeriod,crimeMaskMandateExtendedPeriod,crimeSecondRestrictionsEasedPeriod,crimeExecutiveOrdersDroppedPeriod
0,Removes vict property,12951,11987,26171,10812,11667,10189,18578,18949,2270,5313,52895
1,Stranger,10354,10526,29605,11822,12995,11619,21854,21461,2442,6374,69189
2,Victim knew Suspect,5687,6449,16466,5945,6660,6307,11439,9745,1109,2709,28483
3,Hit-Hit w/ weapon,5465,5379,13078,4484,4872,4754,9231,7606,904,2152,22599
4,Vandalized,5262,5687,13792,5196,5571,5029,9179,8272,1008,2279,22709
...,...,...,...,...,...,...,...,...,...,...,...,...
691,Testing or Probing of Security,0,0,0,0,0,0,0,1,0,0,0
692,Acquires illegal rediological material,0,0,0,0,0,0,0,1,0,0,0
693,Other sensitive materials (susp offer/solicts),0,0,0,0,0,0,0,1,0,0,0
694,T/C - PCF (C) Other Than Driver,0,0,0,0,0,0,0,1,0,0,0


## Query 7
### Get the number of crimes for each period, grouped by the premise in which they occurred

In [202]:
query = """
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>

SELECT ?premisDesc
       (SUM(?crimeCovidStartingPeriod) AS ?crimeCovidStartingPeriod)
       (SUM(?crimeFirstLockdownPeriod) AS ?crimeFirstLockdownPeriod)
       (SUM(?crimeFirstReopeningPeriod) AS ?crimeFirstReopeningPeriod)
       (SUM(?crimeMaskMandatePeriod) AS ?crimeMaskMandatePeriod)
       (SUM(?crimeRestrictionsEasedPeriod) AS ?crimeRestrictionsEasedPeriod)
       (SUM(?crimeVaccineAvailabilityPeriod) AS ?crimeVaccineAvailabilityPeriod)
       (SUM(?crimeRestrictionsDroppedPeriod) AS ?crimeRestrictionsDroppedPeriod)
       (SUM(?crimeVaccineRequirementPeriod) AS ?crimeVaccineRequirementPeriod)
       (SUM(?crimeMaskMandateExtendedPeriod) AS ?crimeMaskMandateExtendedPeriod)
       (SUM(?crimeSecondRestrictionsEasedPeriod) AS ?crimeSecondRestrictionsEasedPeriod)
       (SUM(?crimeExecutiveOrdersDroppedPeriod) AS ?crimeExecutiveOrdersDroppedPeriod)
WHERE {
  ?premis lao:premisDesc ?premisDesc .
  ?crimeEvent lao:hasPremis ?premis ;
              lao:occurredOnDate ?day .
  ?day lao:belongsTo ?period .
  ?period lao:hasLabel ?label .

  BIND(IF(?label = \"Covid Starting Period\", 1, 0) AS ?crimeCovidStartingPeriod)
  BIND(IF(?label = \"First Lockdown Period\", 1, 0) AS ?crimeFirstLockdownPeriod)
  BIND(IF(?label = \"First Reopening Period\", 1, 0) AS ?crimeFirstReopeningPeriod)
  BIND(IF(?label = \"Mask Mandate Period\", 1, 0) AS ?crimeMaskMandatePeriod)
  BIND(IF(?label = \"Restrictions Eased Period\", 1, 0) AS ?crimeRestrictionsEasedPeriod)
  BIND(IF(?label = \"Vaccine Availability Period\", 1, 0) AS ?crimeVaccineAvailabilityPeriod)
  BIND(IF(?label = \"Restrictions Dropped Period\", 1, 0) AS ?crimeRestrictionsDroppedPeriod)
  BIND(IF(?label = \"Vaccine Requirement Period\", 1, 0) AS ?crimeVaccineRequirementPeriod)
  BIND(IF(?label = \"Mask Mandate Extended Period\", 1, 0) AS ?crimeMaskMandateExtendedPeriod)
  BIND(IF(?label = \"Second Restrictions Eased Period\", 1, 0) AS ?crimeSecondRestrictionsEasedPeriod)
  BIND(IF(?label = \"Executive Orders Dropped Period\", 1, 0) AS ?crimeExecutiveOrdersDroppedPeriod)
}
GROUP BY ?premisDesc
ORDER BY DESC(?crimeCovidStartingPeriod)""";

executeQuery(query)

,premisDesc,crimeCovidStartingPeriod,crimeFirstLockdownPeriod,crimeFirstReopeningPeriod,crimeMaskMandatePeriod,crimeRestrictionsEasedPeriod,crimeVaccineAvailabilityPeriod,crimeRestrictionsDroppedPeriod,crimeVaccineRequirementPeriod,crimeMaskMandateExtendedPeriod,crimeSecondRestrictionsEasedPeriod,crimeExecutiveOrdersDroppedPeriod
0,STREET,9457,10018,24171,9459,10120,8972,16655,16088,1857,4295,44370
1,SINGLE FAMILY DWELLING,6907,6910,15542,5954,6599,5894,10604,10098,1116,2998,31674
2,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",4545,4958,11371,4442,4956,4306,7712,7490,868,2101,22405
3,PARKING LOT,2967,2950,6597,2660,2899,2722,4703,4859,582,1259,12337
4,SIDEWALK,1942,1769,4495,1464,1710,1583,2965,2402,276,732,7925
...,...,...,...,...,...,...,...,...,...,...,...,...
293,MTA - ORANGE LINE - VALLEY COLLEGE,0,1,2,0,0,0,0,0,0,0,0
294,MTA - GOLD LINE - LITTLE TOKYO/ARTS DISTRICT,0,0,1,0,0,0,0,0,0,0,0
295,MTA - ORANGE LINE - LAUREL CANYON,0,0,2,0,0,0,1,1,0,0,0
296,DEPT OF DEFENSE FACILITY,0,0,0,0,1,0,0,0,0,0,0


## Query 8
### Get the number of crimes for each period, grouped by the weapon used

In [203]:
query = """
""";

executeQuery(query)

,moDesc,crimeCovidStartingPeriod,crimeFirstLockdownPeriod,crimeFirstReopeningPeriod,crimeMaskMandatePeriod,crimeRestrictionsEasedPeriod,crimeVaccineAvailabilityPeriod,crimeRestrictionsDroppedPeriod,crimeVaccineRequirementPeriod,crimeMaskMandateExtendedPeriod,crimeSecondRestrictionsEasedPeriod,crimeExecutiveOrdersDroppedPeriod
0,Removes vict property,12951,11987,26171,10812,11667,10189,18578,18949,2270,5313,52895
1,Stranger,10354,10526,29605,11822,12995,11619,21854,21461,2442,6374,69189
2,Victim knew Suspect,5687,6449,16466,5945,6660,6307,11439,9745,1109,2709,28483
3,Hit-Hit w/ weapon,5465,5379,13078,4484,4872,4754,9231,7606,904,2152,22599
4,Vandalized,5262,5687,13792,5196,5571,5029,9179,8272,1008,2279,22709
...,...,...,...,...,...,...,...,...,...,...,...,...
691,Testing or Probing of Security,0,0,0,0,0,0,0,1,0,0,0
692,Acquires illegal rediological material,0,0,0,0,0,0,0,1,0,0,0
693,Other sensitive materials (susp offer/solicts),0,0,0,0,0,0,0,1,0,0,0
694,T/C - PCF (C) Other Than Driver,0,0,0,0,0,0,0,1,0,0,0


## Query 9
### Get the number of crimes with PropertyCrime Category for each period, grouped by the date area acroynm between 2020-01-01 and 2022-12-31

In [5]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?date ?acronym (COUNT(?crimeEvent) AS ?count) WHERE {
    ?crimeEvent	rdf:type 				lao:CrimeEvent;
           		lao:occurredInLocation 	?location;
             	lao:occurredOnDate		?day;
              	lao:isOfType			?crime.
    
    ?crime 		rdf:type				?crimeCat.
    ?crimeCat 	rdfs:subClassOf 		lao:Crime.
    
    ?location 	lao:hasLatitude 		?lat;
               	lao:hasLongitude		?lon;
    			lao:belongsToArea		?area.
    
    ?area		lao:areaAcronym			?acronym. 
    ?day 		lao:hasDate				?dt.
    
    FILTER(xsd:date(?dt) < "2022-12-30"^^xsd:date && xsd:date(?dt) > "2021-12-30"^^xsd:date)
    FILTER(?crimeCat = lao:PropertyCrime)
    BIND(SUBSTR(xsd:string(?dt), 1, 10) AS ?date)
}
GROUP BY ?date ?acronym
ORDER BY DESC(xsd:date(?date))
"""
executeQuery(query)

,date,acronym,count
0,2022-12-29,WCH,2
1,2022-12-29,NHL,11
2,2022-12-29,HWD,28
3,2022-12-29,CCY,19
4,2022-12-29,WLK,7
...,...,...,...
12521,2021-12-31,SYL,2
12522,2021-12-31,SLD,1
12523,2021-12-31,HAR,1
12524,2021-12-31,SPD,1


## Query 10
### Get the lat and lon of the crimes with PropertyCrime Category for each period between 2020-01-01 and 2022-12-31


In [6]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?date ?lat ?lon WHERE {
    ?crimeEvent	rdf:type 				lao:CrimeEvent;
           		lao:occurredInLocation 	?location;
             	lao:occurredOnDate		?day;
              	lao:isOfType			?crime.
    
    ?crime 		rdf:type				?crimeCat.
    ?crimeCat 	rdfs:subClassOf 		lao:Crime.
    
    ?location 	lao:hasLatitude 		?lat;
               	lao:hasLongitude		?lon;
    			lao:belongsToArea		?area.
    
    ?area		lao:areaAcronym			?acronym. 
    ?day 		lao:hasDate				?dt.
    
    FILTER(xsd:date(?dt) < "2022-12-30"^^xsd:date && xsd:date(?dt) > "2021-12-30"^^xsd:date)
    FILTER(?crimeCat = lao:PropertyCrime)
    BIND(SUBSTR(xsd:string(?dt), 1, 10) AS ?date)
}
ORDER BY DESC(xsd:date(?date))
"""
executeQuery(query)

,date,lat,lon
0,2022-12-29,33.9599,-118.3801
1,2022-12-29,34.1773,-118.4041
2,2022-12-29,34.1267,-118.3316
3,2022-12-29,34.0524,-118.2467
4,2022-12-29,34.0480,-118.2577
...,...,...,...
88867,2021-12-31,34.0508,-118.2854
88868,2021-12-31,34.0636,-118.3016
88869,2021-12-31,34.0224,-118.2524
88870,2021-12-31,33.9428,-118.2781
